### 역전파 수동화

In [1]:
import numpy as np

class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        
class Function :
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output
    
    def forward(self ,x):
        raise NotImplementedError()
        
    def backward(self, dy):
        raise NotImplementedError()

In [2]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [3]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

#### 순전파 (합성 함수)

In [4]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

#### 역전파 (미분)

In [5]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(y.grad)
print(b.grad)
print(a.grad)
print(x.grad)

1.0
2.568050833375483
3.297442541400256
3.297442541400256


### 역전파 자동화

In [7]:
import numpy as np
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self) # 출력 변수에 창조자 설정
        self.input = input # 입력 변수를 기억(보관)한다.
        self.output = output # 출력도 저장
        return output
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

# 계산 그래프의 노드들을 거꾸로 거슬러 올라간다.
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x


AssertionError: 

In [6]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

In [7]:
class Function:
    def __call__(self, input):
            x = input.data
            y = self.forward(x)
            output = Variable(y)
            output.set_creator(self) # 출력 변수에 창조자 설정
            self.input = input
            self.output = output # 출력 저장
            return output

In [8]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 계산 그래프의 노드들을 거꾸로 거슬러 올라감
# assert는 뒤의 조건이 True가 아니면 AssertError 발생
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

AssertionError: 

In [25]:
y.grad = np.array(1.0)

C = y.creator # 1. 함수를 가져온다.
b = C.input # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad) # 3. 함수의 backward 메서드를 호출한다.

AttributeError: 'NoneType' object has no attribute 'input'

In [26]:
B = b.creator # 1. 함수를 가져온다.
a = B.input # 2. 함수의 입력을 가져온다.
a.grad = B.backward(b.grad) # 3. 함수의 backward 메서드를 호출한다.

AttributeError: 'NoneType' object has no attribute 'input'

In [ ]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

#### backward 메서드 추가

In [30]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator # 1. 함수를 가져온다. 
        if f is not None:
            x = f.input   # 2. 함수의 입력을 가져온다. 
            x.grad = f.backward(self.grad) # 3. 함수의 backward 메서드 호출
            x.backward() # 하나 앞 변수의 backward 메서드를 호출(재귀).

In [31]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파 
y.grad = np.array(1.0)
y.backward()
print(x.grad)

None


#### 반복문을 이용한 구현

In [13]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 함수를 가져온다.
            x, y = f.input, f.output # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad) # backward 메서드를 호출
            
            if x.creator is not None :
                funcs.append(x.creator)

In [14]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파 
y.grad = np.array(1.0)
y.backward()
print(x.grad)

NameError: name 'np' is not defined

In [15]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [16]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))  # 연속 적용

y.grad = np.array(1.0)
y.backward()
print(x.grad)

NameError: name 'np' is not defined

#### backward 메서드 간소화

In [56]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None :
            self.grad = np.ones_like(self.data) # 모든 ndarray 인스턴스를 생성해서, 모든 요소를 1로 채워서 돌려줌
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 함수를 가져온다.
            x, y = f.input, f.output # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad) # backward 메서드를 호출
            
            if x.creator is not None :
                funcs.append(x.creator)

In [57]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


#### ndarray만 취급하기

In [63]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None

In [64]:
x = Variable(np.array(1.0))
x = Variable(None)

x = Variable(1.0)

TypeError: <class 'float'>은(는) 지원하지 않습니다.

In [66]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [68]:
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [1]:
def as_array(x) :
    if np.isscaler(x):
        return np.array(x)
    return x

In [2]:
np.isscaler(np.float64(1.0))

NameError: name 'np' is not defined